In [1]:
from constants import constants
import open3d as o3d
import numpy as np
import scipy.sparse as sp
mesh = o3d.io.read_triangle_mesh("sound_board.stl")

vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
normals = np.asarray(mesh.vertex_normals)

vertices, indices, inverse_indices = np.unique(vertices, axis=0, return_index=True, return_inverse=True)
triangles, normals = inverse_indices[triangles], normals[indices]


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from models import String
string = String(constants)
constants['M_sh'] = string.M_sh
constants['D_h'] = string.D_h
constants['f_sh'] = string.get_f_sh
constants['M_qh'] = string.M_qh

In [3]:
from models import SoundBoard
soundboard = SoundBoard(vertices, triangles, constants)
constants['J_h'] = soundboard.J_h
constants['H_h'] = soundboard.H_h
constants['M_Mh'] = soundboard.M_Mh
constants['M_ph'] = soundboard.M_ph
constants['B_w_h'] = soundboard.B_w_h_transpose.T

SoundBoard Info:
# of vertices: 3508
# of edges: 10200
# of centriods: 6692
# of total points: 20400
# of edges on gamma_f: 112
# of edges on gamma_0: 212


In [4]:
from models import Air
air = Air(constants)
constants['M_pah'] = air.M_pah
constants['Gh'] = air.Gh_transpose.T
constants['M_ah'] = air.M_ah

In [5]:
from models import Gamma
gamma = Gamma(vertices, normals, triangles)

In [6]:
from models import get_B_gamma_h
constants['B_gamma_h'] = get_B_gamma_h(air, gamma)

In [7]:
variables = {
    # variables on the string
    'v_s': np.zeros((constants['Ns']-1)),    # v_s is piecewise constant on the string
    'q': np.zeros((constants['Ns'])),         # q_s is piecewise linear on the string

    # variables on the soundboard
    'v_p': np.zeros((soundboard.num_vertices + soundboard.num_edges + soundboard.num_centers)),
    'M_p': np.zeros((soundboard.num_vertices + soundboard.num_edges + soundboard.num_centers) * 3),

    # variables in the air
    'v_a': np.zeros(3 * (air.resolution[0] * air.resolution[1] * air.resolution[2]) + air.resolution[0] * air.resolution[1] + air.resolution[1] * air.resolution[2] + air.resolution[0] * air.resolution[2]),
    'p_a': np.zeros(air.resolution[0] * air.resolution[1] * air.resolution[2]),
    
    # variables on the gittar body
    'lambda': np.zeros((len(vertices))),
}


for k, v in variables.items():
    print(k, v.shape, sep='\t')
print()

print(f"{constants['M_sh'].shape = }")
print(f"{constants['M_qh'].shape = }")
print(f"{constants['M_Mh'].shape = }")
print(f"{constants['M_ph'].shape = }")
print(f"{constants['M_ah'].shape = }")
print(f"{constants['M_pah'].shape = }")
print(f"{constants['D_h'].shape = }")
print(f"{constants['J_h'].shape = }")
print(f"{constants['H_h'].shape = }")
print(f"{constants['B_w_h'].shape = }")
print(f"{constants['Gh'].shape = }")


v_s	(99,)
q	(100,)
v_p	(20400,)
M_p	(61200,)
v_a	(382500,)
p_a	(125000,)
lambda	(44761,)

constants['M_sh'].shape = (99, 99)
constants['M_qh'].shape = (100, 100)
constants['M_Mh'].shape = (61200, 61200)
constants['M_ph'].shape = (20400, 20400)
constants['M_ah'].shape = (382500, 382500)
constants['M_pah'].shape = (125000, 125000)
constants['D_h'].shape = (99, 100)
constants['J_h'].shape = (20400, 100)
constants['H_h'].shape = (61200, 20400)
constants['B_w_h'].shape = (44761, 20400)
constants['Gh'].shape = (382500, 125000)


In [8]:
import pickle
pickle.dump((constants, (string, soundboard, air, gamma)), open('constants.pkl', 'wb'))